# CAI Claude Instant

In [31]:
import re
from get_llm_response import get_llm_response, Model


In [32]:
model = Model.claude_instant_v1_latest

num_samples = 5

constitution_rules = [
    "Instructions must explicitly describe the tone that the editor adpots.",
    "Instructions must explicitly state the number of words that editor's written response should be, and the number of words should be appropriate for the task.",
    "Instructions should explicitly reference the article content at the beginning, for example 'Here is an article:\n\n<article>ARTICLE_CONTENT</article>\n\n[Rest of instruction goes here]'.",
    "Instructions which involve multiple steps should be broken down into numbered subtasks.",
    # "Instructions must include explicit examples. For example, if the instruction is to check for spelling errors, it should include an example typo and its correction.",
]
constitution_str = "\n".join(map(lambda x: f"{x[0]+1}. {x[1]}", enumerate(constitution_rules)))


def get_concepts_prompt(n: int):
    return f"Generate a list of {n} tasks that a Copy Editor might do as part of their job, given a written article. The output of each completed task should be written text. Write each task on a separate, numbered line."


def get_naive_response_prompt(concept: str):
    return f"""I have an AI agent which acts as a Copy Editor and I want it to complete the following task:

<task>
{concept}
</task>

The agent only responds with written text. Write a concise instruction for the agent, asking it to complete this task. Don't include any preamble, just respond directly with the instruction for the agent."""


def get_critique_prompt(naive_response: str):
    return f"""Here is an instruction to an AI agent which acts as a Copy Editor:

<instruction>
{naive_response}
</instruction>

There may be some problems with this instruction. In particular, the instruction must abide by the following rules:

<rules>
{constitution_str}
</rules>

List each rule that the instruction breaks. State the rule verbatim, then describe how the instruction breaks the rule.

For example, if the instruction breaks rule 1, you would write:

The instruction breaks the following rules:
1. Rule: {constitution_rules[0]} - Reason: ..."""


def get_rewrite_prompt(naive_response: str, critique: str):
    return f"""Here is an instruction to an AI agent which acts as a Copy Editor:

<instruction>
{naive_response}
</instruction>

The instruction is supposed to follow certain rules, but it breaks them as follows:

<issues>
{critique}
</issues>

Rewrite the instruction to address these issues. Respond directly with the rewritten instruction, without any preamble and without <introduction> tags."""


## Generate concepts

In [33]:
concepts_string = get_llm_response(get_concepts_prompt(num_samples), model, 0.7)
print(concepts_string)


 1. Proofread the article for spelling and grammar mistakes.  

2. Check that all facts and figures are accurate.   

3. Ensure consistency in style, capitalization, abbreviations, and punctuation.

4. Make appropriate changes to the text for clarity, conciseness, and readability. 

5. Mark up the text with any copyedits and provide comments or questions for the author.


In [34]:
def extract_items(text):
    items = []
    lines = text.split('\n')
    for line in lines:
        stripped = line.strip()
        if stripped and re.match(r'^\d+\.\s', stripped):
            items.append(re.sub(r'^\d+\.\s', '', stripped))
    return items


concepts = extract_items(concepts_string)
if len(concepts) != num_samples:
    raise Exception(f"Expected {num_samples} concepts, but got {len(concepts)}")
else:
    print(f"Got {len(concepts)} concepts")


Got 5 concepts


## Generate naive responses

> **Terminology** - here we're prompting an LLM to generate prompts, which is confusing. We'll use the word "prompt" to refer to the initial input, and "response" to refer to the output

In [35]:
def get_naive_response(concept):
    return get_llm_response(get_naive_response_prompt(concept), model, 0.3)


# Concurrently (speeds up openai responses, not possible with anthropic)
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     naive_responses = list(executor.map(lambda x: get_naive_response(
#         x[0], x[1]), [(concept, i) for i, concept in enumerate(concepts)]))

# Sequentially
naive_responses = []
for concept in concepts:
    naive_responses.append(get_naive_response(concept))

tasksResponsesString = "\n\n\n===\n\n\n".join(
    [f"TASK:\n{task}\n\nNAIVE RESPONSE:\n{response}" for task, response in zip(concepts, naive_responses)])
print(tasksResponsesString)


TASK:
Proofread the article for spelling and grammar mistakes.

NAIVE RESPONSE:
 Proofread the article and correct any spelling or grammar mistakes.


===


TASK:
Check that all facts and figures are accurate.

NAIVE RESPONSE:
 Verify that all numbers, dates, facts and statistics within the document are correct.


===


TASK:
Ensure consistency in style, capitalization, abbreviations, and punctuation.

NAIVE RESPONSE:
 Please ensure consistency in style, capitalization, abbreviations, and punctuation throughout the text.


===


TASK:
Make appropriate changes to the text for clarity, conciseness, and readability.

NAIVE RESPONSE:
 Revise the text for clarity, conciseness and readability.


===


TASK:
Mark up the text with any copyedits and provide comments or questions for the author.

NAIVE RESPONSE:
 Copyedit the text and provide inline edits and comments indicating any issues.


Most of these responses are slightly more detailed rewrites of the corresponding task. This is a good start, but ultimately not very useful and doesn't demonstrate most of the good practices in prompt design.

## Critique

Here's what an example prompt looks like:

In [36]:
print(get_critique_prompt(naive_responses[0]))


Here is an instruction to an AI agent which acts as a Copy Editor:

<instruction>
 Proofread the article and correct any spelling or grammar mistakes.
</instruction>

There may be some problems with this instruction. In particular, the instruction must abide by the following rules:

<rules>
1. Instructions must explicitly describe the tone that the editor adpots.
2. Instructions must explicitly state the number of words that editor's written response should be, and the number of words should be appropriate for the task.
3. Instructions should explicitly reference the article content at the beginning, for example 'Here is an article:

<article>ARTICLE_CONTENT</article>

[Rest of instruction goes here]'.
4. Instructions which involve multiple steps should be broken down into numbered subtasks.
</rules>

List each rule that the instruction breaks. State the rule verbatim, then describe how the instruction breaks the rule.

For example, if the instruction breaks rule 1, you would write:

T

Now let's execute the critique prompt on each of the naive responses:

In [37]:
def get_critique(naive_response):
    return get_llm_response(get_critique_prompt(naive_response), model, 0)

# Concurrently:
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     critiques = list(executor.map(lambda x: get_critique(x[0], x[1]), [
#                      (response, i) for i, response in enumerate(naive_responses)]))


# Sequentially:
critiques = []
for naive_response in naive_responses:
    critiques.append(get_critique(naive_response))

responseCritiqueString = "\n\n\n===\n\n\n".join(
    [f"NAIVE RESPONSE:\n{response}\n\nCRITIQUE:\n{critique}" for response, critique in zip(naive_responses, critiques)])
print(responseCritiqueString)


NAIVE RESPONSE:
 Proofread the article and correct any spelling or grammar mistakes.

CRITIQUE:
 The instruction breaks the following rules:

1. Rule: Instructions must explicitly describe the tone that the editor adpots.
Reason: The instruction does not specify the tone that the editor should adopt when proofreading and correcting the article.

2. Rule: Instructions must explicitly state the number of words that editor's written response should be, and the number of words should be appropriate for the task.
Reason: The instruction does not specify the number of words that the editor's corrections should contain.

3. Rule: Instructions should explicitly reference the article content at the beginning, for example 'Here is an article:
<article>ARTICLE_CONTENT</article> 
[Rest of instruction goes here].'
Reason: The instruction does not reference any specific article content.

4. Rule: Instructions which involve multiple steps should be broken down into numbered subtasks.    
Reason: The 

## Rewrite

Here's what an example rewrite prompt looks like:

In [38]:
print(get_rewrite_prompt(naive_responses[0], critiques[0]))


Here is an instruction to an AI agent which acts as a Copy Editor:

<instruction>
 Proofread the article and correct any spelling or grammar mistakes.
</instruction>

The instruction is supposed to follow certain rules, but it breaks them as follows:

<issues>
 The instruction breaks the following rules:

1. Rule: Instructions must explicitly describe the tone that the editor adpots.
Reason: The instruction does not specify the tone that the editor should adopt when proofreading and correcting the article.

2. Rule: Instructions must explicitly state the number of words that editor's written response should be, and the number of words should be appropriate for the task.
Reason: The instruction does not specify the number of words that the editor's corrections should contain.

3. Rule: Instructions should explicitly reference the article content at the beginning, for example 'Here is an article:
<article>ARTICLE_CONTENT</article> 
[Rest of instruction goes here].'
Reason: The instructio

Now let's execute the rewrite prompt on each of the naive responses:

In [39]:
def get_rewrite(naive_response, critique):
    return get_llm_response(get_rewrite_prompt(naive_response, critique), model, 0.5)

# Concurrently:
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     rewrites = list(executor.map(lambda x: get_rewrite(x[0], x[1], x[2]), [
#         (response, critique, i) for i, (response, critique) in enumerate(zip(naive_responses, critiques))]))


# Sequentially:
rewrites = []
for i, (naive_response, critique) in enumerate(zip(naive_responses, critiques)):
    rewrites.append(get_rewrite(naive_response, critique))

rewriteString = "\n\n\n===\n\n\n".join(
    [f"NAIVE RESPONSE:\n{response}\n\nREWRITE:\n{rewrite}" for response, rewrite in zip(naive_responses, rewrites)])
print(rewriteString)


NAIVE RESPONSE:
 Proofread the article and correct any spelling or grammar mistakes.

REWRITE:
 Proofread the following 500-1000 word article and correct any spelling or grammar mistakes in under 200 words per correction:

<article>
Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat.  
</article>

1. Check for any spelling mistakes in the article.  
2. Check sentences for correct grammar and punctuation.  
3. Provide concise corrections or comments in under 200 words for any issues you identify.
4. Ensure corrections are polite and respectful in tone.


===


NAIVE RESPONSE:
 Verify that all numbers, dates, facts and statistics within the document are correct.

REWRITE:
 Here is the rewritten instruction:

Here is an article:
<article>ARTICLE_CONTENT</article>

1. Verify that all numbers, dates, facts and sta